In [1]:
import numpy as np
import pandas as pd

In [13]:
import gensim

In [6]:
import os
import requests
import zipfile

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
drive_path = "/content/drive/MyDrive/Year 4/NLP notebooks and data/"

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Year 4/NLP notebooks and data/dataset_cleaned.csv')

# print(df.head())


In [9]:
texts = df['review_text'].astype(str).tolist()

In [10]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

In [ ]:
# Pad sequences - when training
# max_sequence_length = max(len(seq) for seq in sequences)
# data = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

In [11]:
# Step 3: Download and Load Pre-trained FastText Embeddings
# Download the pre-trained FastText model (English)
fasttext_url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz'
fasttext_path = drive_path+'cc.en.300.vec.gz'

In [12]:
if not os.path.exists(fasttext_path):
    print("Downloading FastText embeddings...")
    response = requests.get(fasttext_url)
    with open(fasttext_path, 'wb') as f:
        f.write(response.content)

In [14]:
# Load FastText embeddings
print("Loading FastText model...")
fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(fasttext_path, binary=False)


Loading FastText model...


In [15]:
# Prepare embedding matrix
embedding_dim = 300  # FastText vectors are 300-dimensional
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))


In [16]:
for word, i in word_index.items():
    if word in fasttext_model:
        embedding_matrix[i] = fasttext_model[word]
    else:
        # For words not in the FastText model, you can assign a random vector or leave it as zeros
        embedding_matrix[i] = np.random.normal(size=(embedding_dim,))

In [17]:
# Save the embedding matrix to a file
np.save(drive_path+'FastText_embedding_matrix.npy', embedding_matrix)

In [18]:
# Load the embedding matrix from the file
embedding_matrix = np.load(drive_path+'FastText_embedding_matrix.npy')